In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
%matplotlib inline

import matplotlib.pyplot as plt

## Load In the Data

In [ ]:
skin_df = pd.read_csv('https://raw.githubusercontent.com/smridhiwho/Skin-Detection/main/hmnist_28_28_RGB.csv?token=GHSAT0AAAAAABU7JYFGCJEQMFWQFPHZPXYEYV45Y3Q')

In [ ]:
skin_df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [ ]:
X = skin_df.drop("label", axis=1).values
label = skin_df["label"].values

In [ ]:
X.shape, label.shape

((10015, 2352), (10015,))

## Scaling and Split Data into Train, Validation and Test set

In [ ]:
X_mean = np.mean(X)
X_std = np.std(X)

X = (X - X_mean)/X_std

In [ ]:
X_train_orig, X_test, y_train_orig, y_test = train_test_split(X, label, test_size=0.1,random_state=0)

In [ ]:
X_train_orig.shape, X_test.shape, y_train_orig.shape, y_test.shape

((9013, 2352), (1002, 2352), (9013,), (1002,))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_orig, y_train_orig, test_size=0.2, random_state=1)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((7210, 2352), (1803, 2352), (7210,), (1803,))

## Reshape the Data to Input in CNN

In [ ]:
X_train = X_train.reshape(X_train.shape[0], *(28, 28, 3))
X_val = X_val.reshape(X_val.shape[0], *(28, 28, 3))
X_test = X_test.reshape(X_test.shape[0], *(28, 28, 3))

In [ ]:
X_train.shape, X_val.shape, X_test.shape

((7210, 28, 28, 3), (1803, 28, 28, 3), (1002, 28, 28, 3))

In [ ]:
y_train.shape

(7210,)

In [ ]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [ ]:
y_train.shape, y_val.shape, y_test.shape

((7210, 7), (1803, 7), (1002, 7))

## CNN Model

In [ ]:
# Our input feature map is 28x28x3: 28x28 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(28, 28, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu', padding='same')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 28 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Convolution2D(28, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model
model = Model(img_input, output)

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 28)          8092

## Define Data Generator for Data Augmentation and Learning Rate Adaptive Reduction to Prevent Overfitting

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
batch_size = 64
epochs = 30
history = model.fit(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

Epoch 1/30
112/112 [==============================] - 5s 33ms/step - loss: 1.0229 - accuracy: 0.6591 - val_loss: 0.9410 - val_accuracy: 0.6595 - lr: 0.0010
Epoch 2/30
112/112 [==============================] - 3s 30ms/step - loss: 0.8873 - accuracy: 0.6804 - val_loss: 0.8756 - val_accuracy: 0.6883 - lr: 0.0010
Epoch 3/30
112/112 [==============================] - 3s 30ms/step - loss: 0.8260 - accuracy: 0.6970 - val_loss: 0.8235 - val_accuracy: 0.7033 - lr: 0.0010
Epoch 4/30
112/112 [==============================] - 3s 30ms/step - loss: 0.7961 - accuracy: 0.7061 - val_loss: 0.7943 - val_accuracy: 0.7127 - lr: 0.0010
Epoch 5/30
112/112 [==============================] - 3s 30ms/step - loss: 0.7818 - accuracy: 0.7120 - val_loss: 0.7704 - val_accuracy: 0.7182 - lr: 0.0010
Epoch 6/30
112/112 [==============================] - 3s 30ms/step - loss: 0.7501 - accuracy: 0.7243 - val_loss: 0.7820 - val_accuracy: 0.7171 - lr: 0.0010
Epoch 7/30
112/112 [==============================] - 3s 29ms/st

In [ ]:
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))
model.save("model.h5")

57/57 [==============================] - 0s 3ms/step - loss: 0.6867 - accuracy: 0.7454
Validation: accuracy = 0.745424  ;  loss_v = 0.686714
Test: accuracy = 0.724551  ;  loss = 0.715123
